# pySCENIC protocol: PBMC10k

September 2019

Dataset: *10k PBMCs from a Healthy Donor* available from 10x Genomics ([here](https://support.10xgenomics.com/single-cell-gene-expression/datasets/3.0.0/pbmc_10k_v3)).

In [2]:
# import dependencies
import os
import numpy as np
import pandas as pd
import scanpy as sc
import loompy as lp
from MulticoreTSNE import MulticoreTSNE as TSNE

## SCENIC steps

### STEP 1: Gene regulatory network inference, and generation of co-expression modules
#### Phase Ia: GRN inference using the GRNBoost2 algorithm

For this step the CLI version of SCENIC is used. This step can be deployed on an High Performance Computing system. We use the counts matrix (without log transformation or further processing) from the loom file we wrote earlier.
_Output:_ List of adjacencies between a TF and its targets stored in `ADJACENCIES_FNAME`.

In [3]:
# transcription factors list

f_loom_path_scenic = "/node210data/gyeongdaekim/pyscenic/cnu_lung/cd8.loom"
f_tfs = "/node210data/gyeongdaekim/pyscenic/hs_hgnc_tfs.txt"   # human
f_out_file = "/node210data/gyeongdaekim/pyscenic/cnu_lung/cd8.tsv"

In [ ]:
!pyscenic grn {f_loom_path_scenic} {f_tfs} -o {f_out_file} --num_workers 9


2024-03-13 19:43:33,635 - pyscenic.cli.pyscenic - INFO - Loading expression matrix.

2024-03-13 19:44:10,739 - pyscenic.cli.pyscenic - INFO - Inferring regulatory networks.
/home/gyeongdaekim/anaconda3/envs/pyscenic/lib/python3.7/site-packages/distributed/node.py:161: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38519 instead
  f"Port {expected} is already in use.\n"
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the child process.
Numba: Attempted to fork from a non-main thread, the TBB library may be in an invalid state in the c

read in the adjacencies matrix:

In [4]:
adjacencies = pd.read_csv("/node210data/gyeongdaekim/pyscenic/thyroid/new_M1_M2/m2_cd163_po.tsv", index_col=False, sep='\t')

In [5]:
adjacencies.head()

,TF,target,importance
0,RPL35,RPL36,211.385697
1,YBX1,FTL,210.755757
2,RPS4X,RPL26,209.175938
3,YBX1,SH3BGRL3,204.886261
4,RPL35,RPL36A,199.054813


### STEP 2-3: Regulon prediction aka cisTarget from CLI

For this step the CLI version of SCENIC is used. This step can be deployed on an High Performance Computing system.

_Output:_ List of adjacencies between a TF and its targets stored in `MOTIFS_FNAME`.

locations for ranking databases, and motif annotations:

In [6]:
import glob
# ranking databases
f_db_glob = "/node210data/gyeongdaekim/pyscenic/rcistarget/*.feather"
f_db_names = ' '.join( glob.glob(f_db_glob) )

# motif databases
f_motif_path = "/node210data/gyeongdaekim/pyscenic/rcistarget/motifs-v9-nr.hgnc-m0.001-o0.0.tbl"


# loom file directory
# loom file directory
f_loom_path_scenic = "/node210data/gyeongdaekim/pyscenic/thyroid/new_M1_M2/m2_cd163_po.loom"

f_out_adj_path = "/node210data/gyeongdaekim/pyscenic/thyroid/new_M1_M2/m2_cd163_po.tsv"

f_out_reg_path = "/node210data/gyeongdaekim/pyscenic/thyroid/new_M1_M2/m2_cd163_po_reg.csv"

Here, we use the `--mask_dropouts` option, which affects how the correlation between TF and target genes is calculated during module creation. It is important to note that prior to pySCENIC v0.9.18, the default behavior was to mask dropouts, while in v0.9.18 and later, the correlation is performed using the entire set of cells (including those with zero expression). When using the `modules_from_adjacencies` function directly in python instead of via the command line, the `rho_mask_dropouts` option can be used to control this.

In [7]:
!pyscenic ctx {f_out_adj_path} \
    {f_db_names} \
    --annotations_fname {f_motif_path} \
    --expression_mtx_fname {f_loom_path_scenic} \
    --output {f_out_reg_path} \
    --mask_dropouts \
    --num_workers 12

/home/gyeongdaekim/anaconda3/lib/python3.7/site-packages/dask/dataframe/utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm

2022-10-06 22:52:55,112 - pyscenic.cli.pyscenic - INFO - Creating modules.

2022-10-06 22:52:56,874 - pyscenic.cli.pyscenic - INFO - Loading expression matrix.

2022-10-06 22:53:18,300 - pyscenic.utils - INFO - Calculating Pearson correlations.

2022-10-06 22:53:19,295 - pyscenic.utils - WARNING - Note on correlation calculation: the default behaviour for calculating the correlations has changed after pySCENIC verion 0.9.16. Previously, the default was to calculate the correlation between a TF and target gene using only cells with non-zero expression values (mask_dropouts=True). The current default is now to use all cells to match the behavior of the R verision of SCENIC. The original settings can be retained by setting 'rho_mask_dropouts=True' in the modu

[####                                    ] | 11% Completed |  2min 24.6s
2022-10-06 22:58:07,240 - pyscenic.transform - WARNING - Less than 80% of the genes in Regulon for ZBTB26 could be mapped to hg19-500bp-upstream-7species.mc9nr. Skipping this module.
[####                                    ] | 12% Completed |  2min 26.8s
2022-10-06 22:58:09,365 - pyscenic.transform - WARNING - Less than 80% of the genes in Regulon for ZNF254 could be mapped to hg19-500bp-upstream-7species.mc9nr. Skipping this module.
[######                                  ] | 15% Completed |  2min 33.0s
2022-10-06 22:58:15,626 - pyscenic.transform - WARNING - Less than 80% of the genes in Regulon for AHCTF1 could be mapped to hg19-500bp-upstream-7species.mc9nr. Skipping this module.
[######                                  ] | 15% Completed |  2min 33.4s
2022-10-06 22:58:15,937 - pyscenic.transform - WARNING - Less than 80% of the genes in Regulon for PEG3 could be mapped to hg19-500bp-upstream-7species.mc9nr. 

[########                                ] | 20% Completed |  3min  7.2s
2022-10-06 22:58:49,728 - pyscenic.transform - WARNING - Less than 80% of the genes in Regulon for ZSCAN25 could be mapped to hg19-500bp-upstream-7species.mc9nr. Skipping this module.

2022-10-06 22:58:49,813 - pyscenic.transform - WARNING - Less than 80% of the genes in CEBPZ could be mapped to hg19-500bp-upstream-7species.mc9nr. Skipping this module.
[########                                ] | 20% Completed |  3min  7.3s
2022-10-06 22:58:49,884 - pyscenic.transform - WARNING - Less than 80% of the genes in CERS2 could be mapped to hg19-500bp-upstream-7species.mc9nr. Skipping this module.
[########                                ] | 20% Completed |  3min  7.5s
2022-10-06 22:58:50,082 - pyscenic.transform - WARNING - Less than 80% of the genes in Regulon for ZSCAN5A could be mapped to hg19-500bp-upstream-7species.mc9nr. Skipping this module.
[########                                ] | 20% Completed |  3min  7.6s

[########                                ] | 21% Completed |  3min 15.6s
2022-10-06 22:58:58,152 - pyscenic.transform - WARNING - Less than 80% of the genes in HDAC3 could be mapped to hg19-500bp-upstream-7species.mc9nr. Skipping this module.
[########                                ] | 21% Completed |  3min 15.7s
2022-10-06 22:58:58,244 - pyscenic.transform - WARNING - Less than 80% of the genes in HES1 could be mapped to hg19-500bp-upstream-7species.mc9nr. Skipping this module.
[########                                ] | 21% Completed |  3min 15.8s
2022-10-06 22:58:58,335 - pyscenic.transform - WARNING - Less than 80% of the genes in HES4 could be mapped to hg19-500bp-upstream-7species.mc9nr. Skipping this module.

2022-10-06 22:58:58,365 - pyscenic.transform - WARNING - Less than 80% of the genes in Regulon for ZNF682 could be mapped to hg19-500bp-upstream-7species.mc9nr. Skipping this module.
[########                                ] | 21% Completed |  3min 15.9s
2022-10-06 22:58

[#########                               ] | 23% Completed |  3min 22.2s
2022-10-06 22:59:04,759 - pyscenic.transform - WARNING - Less than 80% of the genes in MLX could be mapped to hg19-500bp-upstream-7species.mc9nr. Skipping this module.
[#########                               ] | 23% Completed |  3min 22.3s
2022-10-06 22:59:04,829 - pyscenic.transform - WARNING - Less than 80% of the genes in MLXIP could be mapped to hg19-500bp-upstream-7species.mc9nr. Skipping this module.

2022-10-06 22:59:04,904 - pyscenic.transform - WARNING - Less than 80% of the genes in MRPL1 could be mapped to hg19-500bp-upstream-7species.mc9nr. Skipping this module.
[#########                               ] | 23% Completed |  3min 22.4s
2022-10-06 22:59:04,980 - pyscenic.transform - WARNING - Less than 80% of the genes in MRPL2 could be mapped to hg19-500bp-upstream-7species.mc9nr. Skipping this module.
[#########                               ] | 23% Completed |  3min 22.6s
2022-10-06 22:59:05,171 - pys

...


2022-10-06 23:05:31,885 - pyscenic.transform - WARNING - Less than 80% of the genes in STUB1 could be mapped to hg19-tss-centered-10kb-7species.mc9nr. Skipping this module.
[################################        ] | 82% Completed |  9min 49.4s
2022-10-06 23:05:31,980 - pyscenic.transform - WARNING - Less than 80% of the genes in SUCLG1 could be mapped to hg19-tss-centered-10kb-7species.mc9nr. Skipping this module.

2022-10-06 23:05:32,012 - pyscenic.transform - WARNING - Less than 80% of the genes in ZNF510 could be mapped to hg19-tss-centered-10kb-7species.mc9nr. Skipping this module.
[################################        ] | 82% Completed |  9min 49.5s
2022-10-06 23:05:32,081 - pyscenic.transform - WARNING - Less than 80% of the genes in ZNF513 could be mapped to hg19-tss-centered-10kb-7species.mc9nr. Skipping this module.

2022-10-06 23:05:32,166 - pyscenic.transform - WARNING - Less than 80% of the genes in ZNF516 could be mapped to hg19-tss-centered-10kb-7species.mc9nr. Skip

[#################################       ] | 83% Completed |  9min 52.6s
2022-10-06 23:05:35,122 - pyscenic.transform - WARNING - Less than 80% of the genes in MAGOH could be mapped to hg19-tss-centered-10kb-7species.mc9nr. Skipping this module.
[#################################       ] | 83% Completed |  9min 52.7s
2022-10-06 23:05:35,221 - pyscenic.transform - WARNING - Less than 80% of the genes in MAPK1 could be mapped to hg19-tss-centered-10kb-7species.mc9nr. Skipping this module.
[#################################       ] | 83% Completed |  9min 52.8s
2022-10-06 23:05:35,364 - pyscenic.transform - WARNING - Less than 80% of the genes in AGMAT could be mapped to hg19-tss-centered-10kb-7species.mc9nr. Skipping this module.
[#################################       ] | 83% Completed |  9min 53.2s
2022-10-06 23:05:35,728 - pyscenic.transform - WARNING - Less than 80% of the genes in ZNF682 could be mapped to hg19-tss-centered-10kb-7species.mc9nr. Skipping this module.
[##############

[##################################      ] | 85% Completed | 10min  5.9s
2022-10-06 23:05:48,453 - pyscenic.transform - WARNING - Less than 80% of the genes in MXD1 could be mapped to hg19-tss-centered-10kb-7species.mc9nr. Skipping this module.

2022-10-06 23:05:48,482 - pyscenic.transform - WARNING - Less than 80% of the genes in ZNF207 could be mapped to hg19-tss-centered-10kb-7species.mc9nr. Skipping this module.
[##################################      ] | 85% Completed | 10min  6.0s
2022-10-06 23:05:48,554 - pyscenic.transform - WARNING - Less than 80% of the genes in ZNF217 could be mapped to hg19-tss-centered-10kb-7species.mc9nr. Skipping this module.
[##################################      ] | 85% Completed | 10min  6.1s
2022-10-06 23:05:48,731 - pyscenic.transform - WARNING - Less than 80% of the genes in ZNF22 could be mapped to hg19-tss-centered-10kb-7species.mc9nr. Skipping this module.
[##################################      ] | 85% Completed | 10min  6.4s
2022-10-06 23:

[##################################      ] | 86% Completed | 10min 14.2s
2022-10-06 23:05:56,776 - pyscenic.transform - WARNING - Less than 80% of the genes in FIP1L1 could be mapped to hg19-tss-centered-10kb-7species.mc9nr. Skipping this module.

2022-10-06 23:05:56,780 - pyscenic.transform - WARNING - Less than 80% of the genes in TGIF1 could be mapped to hg19-tss-centered-10kb-7species.mc9nr. Skipping this module.
[##################################      ] | 86% Completed | 10min 14.3s
2022-10-06 23:05:56,861 - pyscenic.transform - WARNING - Less than 80% of the genes in FLI1 could be mapped to hg19-tss-centered-10kb-7species.mc9nr. Skipping this module.
[##################################      ] | 86% Completed | 10min 14.4s
2022-10-06 23:05:56,992 - pyscenic.transform - WARNING - Less than 80% of the genes in SMARCB1 could be mapped to hg19-tss-centered-10kb-7species.mc9nr. Skipping this module.
[##################################      ] | 86% Completed | 10min 14.5s
2022-10-06 23

[###################################     ] | 88% Completed | 10min 35.6s
2022-10-06 23:06:18,120 - pyscenic.transform - WARNING - Less than 80% of the genes in YY1 could be mapped to hg19-tss-centered-10kb-7species.mc9nr. Skipping this module.
[###################################     ] | 88% Completed | 10min 35.7s
2022-10-06 23:06:18,194 - pyscenic.transform - WARNING - Less than 80% of the genes in ZBED1 could be mapped to hg19-tss-centered-10kb-7species.mc9nr. Skipping this module.
[###################################     ] | 88% Completed | 10min 37.1s
2022-10-06 23:06:19,664 - pyscenic.transform - WARNING - Less than 80% of the genes in RFX1 could be mapped to hg19-tss-centered-10kb-7species.mc9nr. Skipping this module.
[###################################     ] | 89% Completed | 10min 40.2s
2022-10-06 23:06:22,747 - pyscenic.transform - WARNING - Less than 80% of the genes in SRF could be mapped to hg19-tss-centered-10kb-7species.mc9nr. Skipping this module.
[####################

### STEP 4: Cellular enrichment (aka AUCell) from CLI

It is important to check that most cells have a substantial fraction of expressed/detected genes in the calculation of the AUC.
The following histogram gives an idea of the distribution and allows selection of an appropriate threshold.
In this plot, a few thresholds are highlighted, with the number of genes selected shown in red text and the corresponding percentile in parentheses).
See [the relevant section in the R tutorial](https://bioconductor.org/packages/devel/bioc/vignettes/AUCell/inst/doc/AUCell.html#build-gene-expression-rankings-for-each-cell) for more information.

By using the default setting for `--auc_threshold` of `0.05`, we see that **1192** genes are selected for the rankings based on the plot below.

In [8]:
f_loom_path_scenic = "/node210data/gyeongdaekim/pyscenic/thyroid/new_M1_M2/m2_cd163_po.loom"
f_pyscenic_output = "/node210data/gyeongdaekim/pyscenic/thyroid/new_M1_M2/m2_cd163_po_reg_output.loom"
f_out_reg_path = "/node210data/gyeongdaekim/pyscenic/thyroid/new_M1_M2/m2_cd163_po_reg.csv"

In [9]:
!pyscenic aucell \
    {f_loom_path_scenic} \
    {f_out_reg_path} \
    --output {f_pyscenic_output} \
    --num_workers 12

/home/gyeongdaekim/anaconda3/lib/python3.7/site-packages/dask/dataframe/utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm

2022-10-06 23:08:02,224 - pyscenic.cli.pyscenic - INFO - Loading expression matrix.

2022-10-06 23:08:20,612 - pyscenic.cli.pyscenic - INFO - Loading gene signatures.
Create regulons from a dataframe of enriched features.
Additional columns saved: []

2022-10-06 23:08:38,135 - pyscenic.cli.pyscenic - INFO - Calculating cellular enrichment.

2022-10-06 23:09:45,109 - pyscenic.cli.pyscenic - INFO - Writing results to file.


### Visualization of SCENIC's AUC matrix